In [ ]:
using Pkg
cd("../")
Pkg.activate(".")

## General inofrmation

This notebook reproduce all the results for analyzing the reconstruction data in Figure 3.

In [ ]:
using Revise
using PyPlot
using IMRLExploration
using CSV
using DataFrames
using Statistics
using HypothesisTests
using Bootstrap

PyPlot.svg(true)
rcParams = PyPlot.PyDict(PyPlot.matplotlib."rcParams")
rcParams["svg.fonttype"] = "none"
rcParams["pdf.fonttype"] = 42

Colors = ["#004D66","#0350B5","#00CCF5"]
Legends = ["CHF2","CHF3","CHF4"]
σ = 0.2;

## Loading data

In [ ]:
# ------------------------------------------------------------------------------
# Loading data
# ------------------------------------------------------------------------------
Outliers, Long_Subject, Quit_Subject, Data, Goal_type_Set, Sub_Num =
        Read_processed_data(Plotting = true);
# ------------------------------------------------------------------------------
# Loading drawings
# ------------------------------------------------------------------------------
df_graps = coalesce.(CSV.read("data/graphs.csv", DataFrame),0);

## Ground truth

In [ ]:
Names = ["Ts", "P1", "P2", "P3", "P4", "P5", "P6", "X", "S-44"]; Ns=length(Names)
# Function dfg2matg returns the ground-truth graph
#       see src/Functions_for_data.jl
X_truth = dfg2matg()
fig = figure(figsize=(10,8))
X_plot = X_truth;
ax = subplot(1,1,1)
cpfig = ax.imshow(X_plot,vmin=0,vmax=1.0)
for i = 1:Ns
        for j = 1:Ns
                if !isnan(X_plot[i,j])
                        ax.text(j - 1, i - 1, string(round(X_plot[i,j],digits=2)),
                                horizontalalignment="center")
                end
        end
end
fig.colorbar(cpfig, ax=ax)
ax.set_xticks(0:(Ns-1)); ax.set_xticklabels(Names)
ax.set_yticks(0:(Ns-1)); ax.set_yticklabels(Names)
display(fig)


## General statistics of human behavior (Fig 3A1)

In [ ]:
dfg1 = df_graps[Outliers .== 0, :]
Xs = [dfg2matg(dfg1, n; Names = Names) for n = 1:Sub_Num]

fig = figure(figsize=(10,8))
X_plot = mean(Xs); dX_plot = std(Xs) ./ sqrt(Sub_Num)
ax = subplot(1,1,1)
cpfig = ax.imshow(X_plot,vmin=0,vmax=1.0)
for i = 1:Ns
        for j = 1:Ns
                if !isnan(X_plot[i,j])
                        ax.text(j - 1, i - 1.1, string(round(X_plot[i,j],digits=2)),
                                horizontalalignment="center")
                        ax.text(j - 1, i - 0.9, string("+-",round(dX_plot[i,j],digits=2)),
                                horizontalalignment="center")
                end
        end
end
fig.colorbar(cpfig, ax=ax)
ax.set_xticks(0:(Ns-1)); ax.set_xticklabels(Names)
ax.set_yticks(0:(Ns-1)); ax.set_yticklabels(Names)
display(fig)


In [ ]:
# ------------------------------------------------------------------------------
# Number of participants who constructed the entire path from the traps to P6
# ------------------------------------------------------------------------------
X_truth2goal = ldfg2matg2goal(); # all links from traps to state 6
fig = figure(figsize=(10,8))
X_plot = X_truth2goal
ax = subplot(1,1,1)
cpfig = ax.imshow(X_plot,vmin=0,vmax=1.0)
for i = 1:Ns
        for j = 1:Ns
                if !isnan(X_plot[i,j])
                        ax.text(j - 1, i - 1, string(round(X_plot[i,j],digits=2)),
                                horizontalalignment="center")
                end
        end
end
fig.colorbar(cpfig, ax=ax)
ax.set_xticks(0:(Ns-1)); ax.set_xticklabels(Names)
ax.set_yticks(0:(Ns-1)); ax.set_yticklabels(Names)
display(fig)

Fraction2Goal = [matg_2goalscore(X_truth2goal, Xs[n]) for n = eachindex(Xs)]
println("---------------------------")
println("Number of participants who reconstructed the complete path to the goal:")
println(string(sum(Fraction2Goal .== 1)) *  " out of " * string(length(Xs)))
println("---------------------------")

## General comparison (Fig 3A2)

In [ ]:
# 2-hub connections
X_neighb = dfg2matg2HubMask()
fig = figure(figsize=(10,8))
X_plot = X_neighb;
ax = subplot(1,1,1)
cpfig = ax.imshow(X_plot,vmin=0,vmax=1.0)
for i = 1:Ns
        for j = 1:Ns
                if !isnan(X_plot[i,j])
                        ax.text(j - 1, i - 1, string(round(X_plot[i,j],digits=2)),
                                horizontalalignment="center")
                end
        end
end
fig.colorbar(cpfig, ax=ax)
ax.set_xticks(0:(Ns-1)); ax.set_xticklabels(Names)
ax.set_yticks(0:(Ns-1)); ax.set_yticklabels(Names)
display(fig)


In [ ]:
# whether 4 was connected to S-44 for each participant
x_4N = [Xs[n][5,9] for n = eachindex(Xs)]

# whether X was connected to S-44 for each participant
x_XN = [Xs[n][8,9] for n = eachindex(Xs)]

# average errore rate for n-hop connections with n > 2 for each participant
X_wrong1 = 1 .- X_truth; # finding all wrong connections
X_wrong1[8,9] = NaN; # removing the wrong connection between X and S-44
X_wrong1[X_wrong1 .== 0] .= NaN; # replacing correct connections by NaN
X_wrong1[X_neighb .== 1] .= NaN; # removing 2-hop connections
x_wrong1 = [mean(      x[isnan.(X_wrong1) .== 0] .* 
                X_wrong1[isnan.(X_wrong1) .== 0]     ) for x = Xs]

# average errore rate for 2-hop for each participant
x_wrong2 = [mean(x[isnan.(X_neighb) .== 0] .* 
                 X_neighb[isnan.(X_neighb) .== 0]) for x = Xs]

fig = figure(figsize=(5,5))
ax = subplot(1,1,1)
XPlots = [x_wrong1, x_wrong2, x_XN, x_4N]
for i = eachindex(XPlots)
    mx_plot = mean(XPlots[i]); 
    dx_plot = std(XPlots[i]) ./ sqrt(length(XPlots[i]))
    ax.bar(i,mx_plot)
    ax.errorbar(i,mx_plot,yerr=dx_plot,color="k",
            linewidth=1,drawstyle="steps",linestyle="",capsize=3)
end
for i = 1:3
    for j = [4]
            Test_result = OneSampleTTest(XPlots[j],XPlots[i])
            println(Test_result)
            pval = pvalue(Test_result)
            logBF = BIC_OneSampleTTest(XPlots[j],XPlots[i])
            logBF = [logBF, -logBF]
            y_p_temp = 0.05 + i*0.07 + j*0.035
            ax.plot([i,j],[1,1] .* y_p_temp, "k")
            ax.text((i+j)/2,y_p_temp+0.01,
                    string("p:",Func_pval_string(pval),
                            ", lBF:",Func_logBF_string(logBF[1]),
                                    ",",Func_logBF_string(logBF[2])),
                    fontsize=12, horizontalalignment="center", rotation=0)
    end
end
ax.set_xlim([0,5]);ax.set_ylim([0,0.5])
ax.set_xticks(1:4); ax.set_xticklabels(["n-hop","2-hop","X-S44","P4-S44"])
ax.set_ylabel("ratio")
tight_layout()

display(fig)

## Reconstruction score (Fig 3B)

In [ ]:
# ------------------------------------------------------------------------------
# Graph accuracy
# ------------------------------------------------------------------------------
x1 = X_truth[:]; x1 = x1[isnan.(x1) .== 0]
scores = [matg_accscore(X_truth, Xs[n]) for n = 1:Sub_Num]
scores_null1 = [matg_accscore(x1, (rand(length(x1)) .> 0.5) .* 1.) 
                                                        for n = 1:1000]
X_plot_sets = vcat([scores[Goal_type_Set .== g] for g = 0:2],
                [scores_null1])
Legends2 = vcat(Legends, ["Random"])
Colors2 = vcat(Colors, ["#778da9"])
fig = figure(figsize=(5,8))
ax = subplot(1,1,1)
for i = eachindex(X_plot_sets)
        x_plot = X_plot_sets[i]
        mx_plot = mean(x_plot); dx_plot = std(x_plot) ./ sqrt(length(x_plot))
        ax.bar(i,mx_plot, color = Colors2[i])
        ax.errorbar(i,mx_plot,yerr=dx_plot,color="k",
                linewidth=1,drawstyle="steps",linestyle="",capsize=3)
        if Legends2[i] == "Random"
                x_plot = x_plot[1:40]
        else
                Test_result = OneSampleTTest(x_plot)
                println(Test_result)
                logBF = BIC_OneSampleTTest(x_plot)
                logBF = [logBF, -logBF]
                println(logBF)
        end
        ax.plot(i .+ 2 .* σ .* (rand(length(x_plot)) .- 0.5),
                x_plot, ".k", alpha = 0.5)
end
ax.plot([0,length(X_plot_sets) + 1],[0,0],"--k")
ax.set_xlim([0,length(X_plot_sets) + 1]); ax.set_ylim([-0.4,1])
ax.set_xticks(1:length(X_plot_sets)); ax.set_xticklabels(Legends2)
ax.set_ylabel("reconstruction score")
tight_layout()
display(fig)

## Statistics of participants who constructed 4-S44 (Fig 3C-D)

In [ ]:
# ------------------------------------------------------------------------------
# Connections of S44 to state 4 vs, data statistics
# ------------------------------------------------------------------------------
Ys = []; YNames = []; Ylims = []
# number of times in S44
Y = [sum(hcat([Data[n].images[i] for i = 1:5]...)[2,:] .== 43) 
                                                for n = 1:Sub_Num]
Y = hcat(Y,Y) .* 1.
push!(Ys, Y); push!(YNames, "#S44"); push!(Ylims, [0,40])
# number transitions from 4 to S44 and vice versa
# indicator of being in S44
Y1 = [ hcat([Data[n].images[i] for i = 1:5]...)[2,:] .== 43 
                                                for n = 1:Sub_Num]
# indicator of being in 4
Y2 = [(hcat([Data[n].images[i] for i = 1:5]...)[2,:] .== 3) .& 
      (hcat([Data[n].images[i] for i = 1:5]...)[1,:] .== 1)
                                                for n = 1:Sub_Num]
# being in 4 before and after S44
Y = [   sum(Y1[n][2:(length(Y1[n]))] .* Y2[n][1:(length(Y1[n])-1)]) +
        sum(Y2[n][2:(length(Y1[n]))] .* Y1[n][1:(length(Y1[n])-1)]) 
        for n = 1:Sub_Num]
Y = hcat(Y,Y) .* 1.
push!(Ys, Y); push!(YNames, "#4-S44"); push!(Ylims, [0,4])


for i = eachindex(Ys)
        Y = Ys[i]
        x_plot = Y[x_4N .== 0,2]; Gx = Goal_type_Set[x_4N .== 0]
        y_plot = Y[x_4N .== 1,2]; Gy = Goal_type_Set[x_4N .== 1]
        
        fig = figure(figsize=(4,5))
        ax = subplot(1,1,1)
        
        mx_plot = mean(x_plot); dx_plot = std(x_plot) ./ sqrt(length(x_plot))
        ax.bar(1,mx_plot,color="#778da9")
        ax.errorbar(1,mx_plot,yerr=dx_plot,color="k",
                linewidth=1,drawstyle="steps",linestyle="",capsize=3)
        my_plot = mean(y_plot); dy_plot = std(y_plot) ./ sqrt(length(y_plot))
        ax.bar(2,my_plot,color="#778da9")
        ax.errorbar(2,my_plot,yerr=dy_plot,color="k",
                linewidth=1,drawstyle="steps",linestyle="",capsize=3)
        for g = 1:3
                ax.plot(1 .+ 2 .* σ .* (rand(sum(Gx .== (g-1))) .- 0.5),
                        x_plot[Gx .== (g-1)], ".", color = Colors[g])
                ax.plot(2 .+ 2 .* σ .* (rand(sum(Gy .== (g-1))) .- 0.5),
                        y_plot[Gy .== (g-1)], ".", color = Colors[g])
        end
        ax.set_xlim([0,3]); ax.set_ylim(Ylims[i])
        ax.set_xticks(1:2); ax.set_xticklabels(["P4-S44=0","P4-S44=1"])
        ax.set_ylabel(YNames[i])
        Test_result = UnequalVarianceTTest(x_plot, y_plot)
        @show Test_result
        pval = pvalue(Test_result)
        logBF = [BIC_UnequalVarianceTTest(x_plot, y_plot),
                -BIC_UnequalVarianceTTest(x_plot, y_plot)]
        ax.set_title(
                "p:" * Func_pval_string(pval) *
                ", lBF:" * Func_logBF_string(logBF[1]) *
                "," * Func_logBF_string(logBF[2]))
        tight_layout()
        display(fig)        
end
